In [157]:
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor,MultiOutputClassifier
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv('train.csv',header=None)
y_train1 = df.iloc[:,294:]
x_train1= df.iloc[:,:294]

#x_train1=x_train1.drop(drop_col_indx,axis=1)

X_train, X_test, y_train, y_test = train_test_split( x_train1, y_train1, test_size=0.2, random_state=42)

#y_test1= MultiOutputRegressor(GradientBoostingRegressor(min_samples_split=25,learning_rate=0.15,max_depth=4, min_samples_leaf=10,random_state=10,n_estimators=22)).fit(X_train, y_train).predict(X_test)

#y_test1= MultiOutputRegressor(SVR(C=60,kernel='linear')).fit(X_train, y_train).predict(X_test)



In [159]:
pipe_svr = Pipeline([('scl', StandardScaler()),
        ('reg', MultiOutputRegressor(SVR(C=150,epsilon=0.1)))])

pipe_svc = Pipeline([('scl', StandardScaler()),
        ('clas', MultiOutputClassifier(SVC(C=100)))])


y_test1=pipe_svr.fit(X_train, y_train).predict(X_test)

#For Rounding the Regressor Output to INt
y_test1=np.around(y_test1, decimals=0)
y_test1=y_test1.astype(np.int64)


#Calculating the Accuracy 
accuracy_score(y_test,y_test1)

0.5902777777777778

##GridSearch CV FOR MULTIOUTPUT Regressor

In [156]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipe_svr = Pipeline([('scl', StandardScaler()),
        ('reg', MultiOutputRegressor(SVR()))])

grid_param_svr = {
    'reg__estimator__C': [50,100,150,175,200,250,300],
    'reg__estimator__epsilon':[0.1]
}

gs_svr = (GridSearchCV(estimator=pipe_svr, 
                      param_grid=grid_param_svr, 
                      cv=10,
                      
                      n_jobs = -1))

gs_svr = gs_svr.fit(x_train1,y_train1)
gs_svr.best_estimator_

/home/dhruvrajrawat/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Pipeline(memory=None,
         steps=[('scl',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('reg',
                 MultiOutputRegressor(estimator=SVR(C=150, cache_size=200,
                                                    coef0=0.0, degree=3,
                                                    epsilon=0.1,
                                                    gamma='auto_deprecated',
                                                    kernel='rbf', max_iter=-1,
                                                    shrinking=True, tol=0.001,
                                                    verbose=False),
                                      n_jobs=None))],
         verbose=False)

In [60]:
pipe_svr.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scl', 'reg', 'scl__copy', 'scl__with_mean', 'scl__with_std', 'reg__estimator__C', 'reg__estimator__cache_size', 'reg__estimator__coef0', 'reg__estimator__degree', 'reg__estimator__epsilon', 'reg__estimator__gamma', 'reg__estimator__kernel', 'reg__estimator__max_iter', 'reg__estimator__shrinking', 'reg__estimator__tol', 'reg__estimator__verbose', 'reg__estimator', 'reg__n_jobs'])

## Feature Selection

In [147]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

regr_multi_RF=MultiOutputRegressor(RandomForestRegressor(min_samples_split=20, min_samples_leaf=10,random_state=10,n_estimators=30))
regr_multi_RF.fit(x_train1,y_train1)

# how many estimators?
len(regr_multi_RF.estimators_)
# 2

regr_multi_RF.estimators_[0].feature_importances_
# array([ 0.4,  0.6])

regr_multi_RF.estimators_[1].feature_importances_
#

array([1.74168595e-03, 1.05714061e-04, 4.47237845e-04, 1.88542075e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.38187468e-03,
       7.10380809e-04, 1.75349918e-03, 3.66167784e-03, 3.21716985e-03,
       2.25133368e-03, 3.51871443e-03, 1.12504393e-02, 6.25346454e-04,
       1.89569958e-05, 8.01500456e-04, 4.37944013e-03, 2.25090511e-02,
       1.94758288e-02, 1.69307396e-02, 2.44624965e-02, 1.91742423e-02,
       4.01106050e-02, 7.43140883e-02, 2.38216901e-02, 1.06523725e-02,
       1.67750549e-03, 2.39082441e-03, 8.13265992e-04, 3.15791385e-05,
       2.06335716e-03, 1.30913389e-03, 7.74175952e-04, 2.71847868e-03,
       0.00000000e+00, 0.00000000e+00, 1.07183599e-03, 2.58502934e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.42590690e-03, 2.94675569e-03, 1.64531257e-04, 7.12012119e-04,
       2.06978423e-03, 3.44279619e-01, 3.13942487e-03, 2.53456964e-03,
       8.72383638e-02, 8.54563218e-03, 6.82139261e-03, 8.04271818e-03,
      

In [148]:
imp_coef = regr_multi_RF.estimators_[0].feature_importances_

In [151]:
drop_col_indx=[]
for col,val in enumerate(imp_coef):
    if val<0.0001:
        drop_col_indx.append(col)

In [152]:
drop_col_indx

[1,
 34,
 65,
 66,
 68,
 69,
 70,
 72,
 88,
 109,
 123,
 126,
 136,
 139,
 140,
 141,
 150,
 151,
 153,
 158,
 159]

In [131]:
x_train1.drop(drop_col_indx,axis=1)

,13,17,18,19,21,24,25,26,27,28,...,284,285,286,287,288,289,290,291,292,293
0,0.792806,0.847413,0.744884,0.764534,0.810743,0.838431,0.751858,0.753316,0.849535,0.303032,...,0.317461,0.189214,0.269183,0.016788,0.013705,0.012601,0.010687,0.010919,0.011375,0.015069
1,0.631642,0.670668,0.677323,0.748249,0.782133,0.611240,0.632528,0.812389,0.637667,0.475344,...,0.078913,0.175919,0.223353,0.041061,0.050408,0.051189,0.050673,0.055661,0.079765,0.097522
2,0.982745,0.850750,0.785884,0.922892,0.944247,0.826791,0.720847,0.876573,0.980914,0.794011,...,0.209348,0.169218,0.337669,0.083091,0.187438,0.532243,0.232948,0.195177,0.221791,0.201402
3,0.766792,0.610776,0.575085,0.472539,0.442304,0.500253,0.507535,0.405099,0.509126,0.366228,...,0.370191,0.084081,0.145852,0.148042,0.211724,0.115635,0.297193,0.254088,0.221800,0.146297
4,0.685375,0.774313,0.639332,0.592065,0.465693,0.507610,0.520886,0.610026,0.649071,0.377131,...,0.442008,0.347993,0.256008,0.166874,0.100561,0.157648,0.129825,0.075116,0.060953,0.022725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,0.740936,0.800728,0.798471,0.699458,0.797164,0.757937,0.605701,0.472117,0.277052,0.848281,...,0.030961,0.036122,0.075832,0.061080,0.077239,0.038433,0.016594,0.007776,0.043936,0.037436
1434,0.640453,0.697983,0.694132,0.687415,0.512566,0.661690,0.700183,0.644097,0.653159,0.457523,...,0.140346,0.347075,0.200204,0.003339,0.003448,0.002265,0.009045,0.032089,0.141872,0.010918
1435,0.802678,0.729252,0.749867,0.782630,0.516204,0.696617,0.723308,0.587297,0.864239,0.379290,...,0.058883,0.004463,0.002236,0.401406,0.736897,0.172458,0.003192,0.030422,0.004995,0.002626
1436,0.582376,0.648720,0.613431,0.548195,0.754202,0.815766,0.809939,0.798278,0.754965,0.731211,...,0.059576,0.134512,0.057485,0.022966,0.018802,0.040643,0.116047,0.039915,0.075365,0.056335


In [ ]:
df_test= pd.DataFrame(data=y_test)
#np.savetxt("foo.csv", y_test, delimiter=",")

df_test.to_csv('prediction.csv',index=False,header=False)